Librabry Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Data Collection

In [2]:
df=pd.read_csv(r"D:\DATA SCIENCE\data science\9. Natural Language Processing\Restaurant_Reviews.tsv",delimiter = '\t')

Exploratory Data Analysis

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Liked   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB
None


In [4]:
print(df.head())

                                              Review  Liked
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1


In [5]:
print(df.isnull().sum())

Review    0
Liked     0
dtype: int64


In [6]:
print(df['Liked'].nunique())

2


Downloading Stopwords

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
a=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agala\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cleaning the Data Using Stopwords, Stemming and Dependency Parsing

In [8]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
print(ps.stem('lovely'))
print(ps.stem('tokenization'))

love
token


In [9]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")

In [10]:
lst1 = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]',' ', df['Review'][i])
    review = review.lower()
    dp = nlp(review)
    dpa = [token.lemma_ for token in dp]
    v = [ps.stem(token) for token in dpa if token not in a]
    v = ' '.join(v)
    lst1.append(v)

Vectorization and Selection of x and y Variable

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(lst1).toarray()
y=df['Liked'].values

Model Selection and Training

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30,random_state=40)

In [13]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

Model Evaluation

In [14]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[130  43]
 [ 20 107]]


In [15]:
cr=classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.87      0.75      0.80       173
           1       0.71      0.84      0.77       127

    accuracy                           0.79       300
   macro avg       0.79      0.80      0.79       300
weighted avg       0.80      0.79      0.79       300



Cross-Validation

In [16]:
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
print('logistic regression accuracy is: ',cross_val_score(log,x,y,cv=10,scoring='accuracy').mean())

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
print('random forest accuracy is: ',cross_val_score(rfc,x,y,cv=10,scoring='accuracy').mean())

from sklearn.svm import SVC
classifier = SVC()
print('svm accuracy is: ',cross_val_score(classifier,x,y,cv=10,scoring='accuracy').mean())

logistic regression accuracy is:  0.7969999999999999
random forest accuracy is:  0.758
svm accuracy is:  0.775


Hyperparameter Tuning

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# Define the hyperparameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0],
              'fit_prior': [True, False]}

# Create the Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Perform grid search with cross-validation
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# Get the best hyperparameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(best_params)
print(best_score)

{'alpha': 1.0, 'fit_prior': False}
0.7785714285714287


In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from scipy.stats import uniform

# Define the hyperparameter distributions
param_dist = {'alpha': uniform(0.1, 2.0),
              'fit_prior': [True, False]}

# Create the Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Perform random search with cross-validation
random_search = RandomizedSearchCV(classifier, param_dist, n_iter=10, cv=5, scoring='accuracy')
random_search.fit(x_train, y_train)

# Get the best hyperparameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_
print(best_params)
print(best_score)

{'alpha': 1.3571938800686731, 'fit_prior': False}
0.7785714285714287
